In [9]:
## importar as bibliotecas
import subprocess
import docker
import time
from docker.errors import NotFound

subprocess.run(["pip", "install", "docker"], check=True)
client = docker.from_env()

time.sleep(2)

In [10]:
## Verifica se a rede existe

def network_exists(network_name):
    try:
        client.networks.get(network_name)
        return True
    except NotFound:
        return False
network_name = "mongo-shard"

if not network_exists(network_name):
    client.networks.create(network_name)
    print(f"Rede '{network_name}' criada com sucesso.")
else:
    print(f"Rede '{network_name}' já existe.")

time.sleep(2)

Rede 'mongo-shard' já existe.


In [15]:
## Cria os containers

container_names=["mongo-config01", "mongo-config02", "mongo-config03"]

for name in container_names:
    subprocess.run(["docker",  "run", "--name", name, "--net", "mongo-shard", "-d", "mongo", "mongod", "--configsvr", "--replSet", "configserver", "--port", "27017"])

time.sleep(2)

FileNotFoundError: [Errno 2] No such file or directory: 'docker'

In [ ]:
## Acessa o shell do container
container = client.containers.get(container_names[0])

command = """
mongosh --eval 'rs.initiate({_id: "configserver", configsvr: true, version: 1, members: [{ _id: 0, host: "mongo-config01:27017"}, { _id: 1, host: "mongo-config02:27017"}, { _id: 2, host: "mongo-config03:27017"}]})'
"""

response_script = container.exec_run(command)

print(response_script.output.decode())

time.sleep(2)

In [ ]:
# Criação dos shards

shards = [
    {
    'name': "shard1",
    'value': "shard01",
    'port': "27018"},
    {
    'name': "shard2",
    'value': "shard02",
    'port': "27019"},
    {
    'name': "shard3",
    'value': "shard03",
    'port': "27020"}
    ]

for shard in shards:
    print(shard)
    sharda = subprocess.run(["docker",  "run", "--name", f"mongo-{shard['name']}a", "--net", "mongo-shard", "-d", "mongo", "mongod", "--port", shard['port'], "--shardsvr", "--replSet", shard['value']])
    shardb = subprocess.run(["docker",  "run", "--name", f"mongo-{shard['name']}b", "--net", "mongo-shard", "-d", "mongo", "mongod", "--port", shard['port'], "--shardsvr", "--replSet", shard['value']])

time.sleep(2)

In [ ]:
## Acessa o shell dos shards

for shard in shards:
   containerA = client.containers.get( f"mongo-{shard['name']}a")
   scriptA = "mongosh --port " + shard['port'] + " --eval \"rs.initiate({_id: '" + shard['value'] + "',version: 1,members: [{ _id: 0, host : 'mongo-" + shard['name'] + "a:" + shard['port'] + "' },{ _id: 1, host : 'mongo-" + shard['name'] + "b:" + shard['port'] + "' }]})\""
   response_scriptA = containerA.exec_run(scriptA)
   print(response_scriptA.output.decode())



In [ ]:
# Cria o roteador

router = subprocess.run([
    "docker", "run", 
    "-p", "27017:27017", 
    "--name", "mongo-router", 
    "--net", "mongo-shard", 
    "-d", "mongo", 
    "mongos", 
    "--port", "27017", 
    "--configdb", "configserver/mongo-config01:27017,mongo-config02:27017,mongo-config03:27017", 
    "--bind_ip_all"
], capture_output=True, text=True)

print(router.stdout)
print(router.stderr)

In [ ]:
## Acessa o shell do roteador
shard_commands = [
    'sh.addShard("shard01/mongo-shard1a:27018")',
    'sh.addShard("shard01/mongo-shard1b:27018")',
    'sh.addShard("shard02/mongo-shard2a:27019")',
    'sh.addShard("shard02/mongo-shard2b:27019")',
    'sh.addShard("shard03/mongo-shard3a:27020")',
    'sh.addShard("shard03/mongo-shard3b:27020")'
]

routerContainer = client.containers.get("mongo-router")

for command in shard_commands:
    response_routerContainer = routerContainer.exec_run(['mongosh', '--eval', command])
    print(response_routerContainer.output.decode())
    time.sleep(2)

# Aguarda alguns segundos antes de prosseguir


In [ ]:
response_routerContainer = routerContainer.exec_run(['mongosh', '--eval', 'sh.status()'])
print(response_routerContainer.output.decode())